In [17]:
import requests

In [18]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [19]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [20]:
import hashlib

In [21]:
def generate_document_id(doc:dict) -> str:
    """Function to take dictionary and return dictionary"""

    combined_string = f"{doc['section']}-{doc["course"]}-{doc['question']}-{doc['text'][:16]}"
    generated_hash = hashlib.md5(combined_string.encode())
    stored_hash = generated_hash.hexdigest()[:16]
    return stored_hash

In [40]:
documents_updated = [{**doc, "document_id": generate_document_id(doc)} for doc in documents]

In [42]:
from collections import defaultdict

In [45]:
hashes = defaultdict(list)

for doc in documents_updated:
    doc_id = doc['document_id']
    hashes[doc_id].append(doc)

In [46]:
len(documents), len(hashes)

(948, 947)

In [49]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

406267ecd3b4eb48 2


In [50]:
hashes['406267ecd3b4eb48']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'document_id': '406267ecd3b4eb48'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'document_id': '406267ecd3b4eb48'}]

In [51]:
import json

In [52]:
with open('documents-with-ids-updated.json', 'wt') as f_out:
    json.dump(documents_updated, f_out, indent=2)

In [53]:
!head documents-with-ids-updated.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "document_id": "12ba9d1bc6ca27c5"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [54]:
prompt_template = """
You emulate a student who's taking our course and asking questions about the course.
Formulate 5 similar questions this student might ask based on a FAQ record which will be shared a json. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. The question generated must be relevant to the existing answer.

The record:

section: {section}
question: {question}
answer: {text}
course: {course}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [61]:
from openai import OpenAI
from openai import RateLimitError
import time

In [56]:
client = OpenAI()

In [67]:
def generate_questions(doc, max_retries=5, initial_delay=1):
    delay = initial_delay
    prompt = prompt_template.format(**doc)
    for i in range(max_retries):
        try:
            response = client.chat.completions.create(
                model='gpt-4o-mini',
                messages=[{"role": "user", "content": prompt}]
            )
        
            json_response = response.choices[0].message.content
            break
        except RateLimitError:
            time.delay((i+1)*delay)
            delay*=2
        except Exception as e:
            print(f"Unpexpected error {e}")
            break
        
    return json_response

In [68]:
from tqdm.auto import tqdm

In [69]:
results = {}

In [70]:
for doc in tqdm(documents_updated):
    doc_id = doc['document_id']
    if doc_id in results:
        continue
    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

In [1]:
import re

In [102]:
results_clean = {}
for k, v in results.items():
    clean_string = re.sub(r'[^a-zA-Z0-9,?\s]', '', v).strip()
    results_clean[k] = [i.strip() for i in clean_string.strip().split(",\n")]

In [104]:
import pickle

In [110]:
with open('results_practice.bin', 'wb') as f_out:
    pickle.dump(results_clean, f_out)

In [111]:
with open('results_practice.bin', 'rb') as f_in:
    results_data = pickle.load(f_in)

In [115]:
final_resuls = []
for doc_id, questions in results_data.items():
    course = hashes[doc_id][0]['course']
    for q in questions:
        final_resuls.append((q, course, doc_id))

In [116]:
import pandas as pd

In [117]:
df = pd.DataFrame(data=final_resuls, columns=['question', 'course', 'document'])
df.to_csv('ground-truth-data-practice.csv', index=False)